In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, AutoPeftModelForCausalLM

/storage/ice1/2/3/ckniffin6/conda_envs/hf/lib/python3.12/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [45]:
from huggingface_hub import login
login()

In [15]:
model = AutoPeftModelForCausalLM.from_pretrained("/home/hice1/ckniffin6/scratch/DL/project/adapters/Lora_rank8").to("cuda")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [16]:
# Load the saved model
modelName = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(modelName)

base_prompt_template: str = """
    You are a financial analyst designed to answer questions about business and finance. Your job is to reply to questions about finance topics and provide advice.

    Instruction: {query}
    
    Output:
"""
    

In [17]:
queries = ["If I take out a $35000 loan with a 5% APR how many $500 payments would it take to pay it back?",
           "Assuming an inital investment of $100,000, an expected cash flow of $7000 per month, an interest rate of 7% and 3 years before the project is complete what is the net present value?",
           "You want to save up over 4 years for you child's college. If the cost of attendance is $100000 and you deposit money each month into a savings account with a 4% annual interest rate, how much do you have to deposit each month?",
           "How does a business compute net income?",
           "What is the rule of 72?",
           "What is CPI and how does it tell us what the inflation rate is?"]    

In [18]:
summary1 = '"As of March 31, 2024, the Company had aggregate outstanding notes of $14,016 million, net of $61 million of issuance costs, with varying maturities (the "Notes"). Of the outstanding balance, $799 million, net of issuance costs, is classified as short-term debt on the Consolidated Balance Sheets. As of December 31, 2023, the Company had aggregate outstanding notes of $14,543 million, net of $65 million of issuance costs. Each of the Notes were issued at par and are senior unsecured obligations of the Company. Interest is payable semi-annually at fixed rates. A portion of the outstanding Notes is denominated in foreign currency (comprised of €5,170 million) and is remeasured into U.S. dollars at each balance sheet date (with remeasurement gain totaling $131 million for the three months ended March 31, 2024)."  Based on the preceding text how much short term debt does the company have?'

In [19]:
summary2 = '"We anticipate our full-year 2024 capital expenditures will be in the range of $30-37 billion, a $2 billion increase of the high end of our prior range. We expect growth will be driven by investments in servers, including both AI and non-AI hardware, and data centers as we ramp up construction on sites with our previously announced new data center architecture. Our updated outlook reflects our evolving understanding of our artificial intelligence (AI) capacity demands as we anticipate what we may need for the next generations of foundational research and product development. While we are not providing guidance for years beyond 2024, we expect our ambitious long-term AI research and product development efforts will require growing infrastructure investments beyond this year." Based on this text what are some risks the that could cause its expenditures to be greater than the predicted $2 billion?'

In [30]:
summary3 = '"Product Liability We recorded liabilities of $596 million and $561 million in Accrued liabilities and Other liabilities at September 30, 2023 and December 31, 2022 for the expected cost of all known product liability claims, plus an estimate of the expected cost for product liability claims that have already been incurred and are expected to be filed in the future for which we are self-insured. It is reasonably possible that our accruals for product liability claims may increase in future periods in material amounts, although we cannot estimate a reasonable range of incremental loss based on currently available information. We believe that any judgment against us involving our products for actual damages will be adequately covered by our recorded accruals and, where applicable, excess liability insurance coverage." How much did the company accrue in total liabilities according to the preceeding text?'

In [21]:
query = base_prompt_template.format(query=queries[0])

modelInput = tokenizer(query, return_tensors="pt").to("cuda")

outputs = model.generate(**modelInput, max_new_tokens = 500, do_sample = True)
print(tokenizer.batch_decode(outputs)[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> 
    You are a financial analyst designed to answer questions about business and finance. Your job is to reply to questions about finance topics and provide advice.

    Instruction: If I take out a $35000 loan with a 5% APR how many $500 payments would it take to pay it back?
    
    Output:
<s> As you've discovered, the interest is compounding by the month, so 5% of $35k is about how much you'll need to add on.  Let's use the exact number, 1825.00 (5% monthly interest rate over 36 months)  Your monthly payment with no interest is $1020.60. Adding in interest, your monthly payment is $1182.64 (rounded). You'll pay off this amount (just interest, and 10$/mo for the loan) in month 12.   Your second payment is towards loan, interest, and principal. So $1025 is paid towards the principal and interest, and $78 is interest that you accrue monthly.  The third payment will pay down the principal even more, so will be higher than the second.   I think this is a good time to start tracking

In [22]:
query = base_prompt_template.format(query=queries[1])

modelInput = tokenizer(query, return_tensors="pt").to("cuda")

outputs = model.generate(**modelInput, max_new_tokens = 500, do_sample = True)
print(tokenizer.batch_decode(outputs)[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> 
    You are a financial analyst designed to answer questions about business and finance. Your job is to reply to questions about finance topics and provide advice.

    Instruction: Assuming an inital investment of $100,000, an expected cash flow of $7000 per month, an interest rate of 7% and 3 years before the project is complete what is the net present value?
    
    Output:
<s> The net present value of the investment is $601,784.65. This assumes an initial investment of $100,000, a monthly cash flow of $7,000, an interest rate of 7% and 3 years before the project is complete. The formula used to calculate this is: NPV = - C0 + (CF / (1+r) ^1) + (CF / (1+r) ^2) + (CF / (1+r) ^3) = -$100,000 + $7,000 / (1+0.07) ^1 + $7,000 / (1+0.07) ^2 + $7,000 / (1+0.07) ^3 = $601,784.65. Therefore, the net present value of this investment is $601,784.65, which makes it a beneficial investment.  Please note: the assumption here is that the investor receives equal monthly cash flow every month 

In [23]:
query = base_prompt_template.format(query=queries[2])

modelInput = tokenizer(query, return_tensors="pt").to("cuda")

outputs = model.generate(**modelInput, max_new_tokens = 500, do_sample = True)
print(tokenizer.batch_decode(outputs)[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> 
    You are a financial analyst designed to answer questions about business and finance. Your job is to reply to questions about finance topics and provide advice.

    Instruction: You want to save up over 4 years for you child's college. If the cost of attendance is $100000 and you deposit money each month into a savings account with a 4% annual interest rate, how much do you have to deposit each month?
    
    Output:
<s> To have $100,000 after 4 years at a 4% annual interest rate, the amount you need to deposit monthly is around $2,024  In order to calculate the amount you should deposit monthly, we need two things: the cost of attendance and the compound interest rate of the savings account. Cost of attendance ($100,000) x ($1.04)^4 = $100,000   (1+r/100)4 = 1.04  Where r is the interest rate   100000x(1+0.04)^4 = 100000   1.04^4 = 1.216  Rearranging for r   4(r/100) = 0.1216  Where r is the interest rate r = 12.16%   Deposit amount needed per month to reach $100,000 at end 

In [24]:
query = base_prompt_template.format(query=queries[3])

modelInput = tokenizer(query, return_tensors="pt").to("cuda")

outputs = model.generate(**modelInput, max_new_tokens = 500, do_sample = True)
print(tokenizer.batch_decode(outputs)[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> 
    You are a financial analyst designed to answer questions about business and finance. Your job is to reply to questions about finance topics and provide advice.

    Instruction: How does a business compute net income?
    
    Output:
<s> You've got most things right, I just needed to make few edits:  Cash Flow from Operations is Net Income plus Difference between the Operating Assets and Operating Liabilities. Cash Flow from Operations is similar but not equal to the Net Income. You missed Operating Expenses and it was the reason the numbers on your answer were wrong. To understand why Operating Expenses need to be taken out from Net Income, look at this simple example: you have operating expenses like rent, salaries, etc. and net income in a given period. You sold a car for 8000, you received 5000 from rent and you paid 3000 for salaries. You still have operating cash in your company. However, if you then used all this cash to buy a new car for 6000, you will still have the 

In [25]:
query = base_prompt_template.format(query=queries[4])

modelInput = tokenizer(query, return_tensors="pt").to("cuda")

outputs = model.generate(**modelInput, max_new_tokens = 500, do_sample = True)
print(tokenizer.batch_decode(outputs)[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> 
    You are a financial analyst designed to answer questions about business and finance. Your job is to reply to questions about finance topics and provide advice.

    Instruction: What is the rule of 72?
    
    Output:
<s> The Rule of 72 is often used as a shortcut by novice investors to compute the annual rate of return necessary to double one's investment in one year.  Simply, the Rule of 72 states that a fixed annual rate of return of 72 divided by the number of years will double an investment. As such, a 10% return is double in 7.2 years, 20% returns are double in 3.6 years, and so on. It's more useful for quick approximations. A more accurate formula, based on compound interest, is to take the number of years, divide by the rate, and take the natural log: Years for Doubling =  ln(2) / Rate. If you are doubling your investment, it is called a doubling period, as mentioned above in the first paragraph. Doubling every seven years is considered the standard rate of return, wh

In [26]:
query = base_prompt_template.format(query=queries[5])

modelInput = tokenizer(query, return_tensors="pt").to("cuda")

outputs = model.generate(**modelInput, max_new_tokens = 500, do_sample = True)
print(tokenizer.batch_decode(outputs)[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> 
    You are a financial analyst designed to answer questions about business and finance. Your job is to reply to questions about finance topics and provide advice.

    Instruction: What is CPI and how does it tell us what the inflation rate is?
    
    Output:
<s> The Consumer Price Index is basically the average price of a "basket of goods" or products that people typically buy. This list contains products of various food, housing, clothes, etc. So basically, if a gallon of milk is $3 now, but $4 a year prior, the CPI will take that into account and factor it into their index, hence showing that there is inflation since the price has increased. If they used the same list of products, but kept the prices constant for a certain period of time while also keeping in mind products that were added and removed from the list, and how those would affect prices, this would be a general idea of what the index would look like. The CPI is a major factor in the stock market as well as the bo

In [27]:
query = base_prompt_template.format(query=summary1)

modelInput = tokenizer(query, return_tensors="pt").to("cuda")

outputs = model.generate(**modelInput, max_new_tokens = 500, do_sample = True)
print(tokenizer.batch_decode(outputs)[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> 
    You are a financial analyst designed to answer questions about business and finance. Your job is to reply to questions about finance topics and provide advice.

    Instruction: "As of March 31, 2024, the Company had aggregate outstanding notes of $14,016 million, net of $61 million of issuance costs, with varying maturities (the "Notes"). Of the outstanding balance, $799 million, net of issuance costs, is classified as short-term debt on the Consolidated Balance Sheets. As of December 31, 2023, the Company had aggregate outstanding notes of $14,543 million, net of $65 million of issuance costs. Each of the Notes were issued at par and are senior unsecured obligations of the Company. Interest is payable semi-annually at fixed rates. A portion of the outstanding Notes is denominated in foreign currency (comprised of €5,170 million) and is remeasured into U.S. dollars at each balance sheet date (with remeasurement gain totaling $131 million for the three months ended March 31, 2

In [28]:
query = base_prompt_template.format(query=summary2)

modelInput = tokenizer(query, return_tensors="pt").to("cuda")

outputs = model.generate(**modelInput, max_new_tokens = 500, do_sample = True)
print(tokenizer.batch_decode(outputs)[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> 
    You are a financial analyst designed to answer questions about business and finance. Your job is to reply to questions about finance topics and provide advice.

    Instruction: "We anticipate our full-year 2024 capital expenditures will be in the range of $30-37 billion, a $2 billion increase of the high end of our prior range. We expect growth will be driven by investments in servers, including both AI and non-AI hardware, and data centers as we ramp up construction on sites with our previously announced new data center architecture. Our updated outlook reflects our evolving understanding of our artificial intelligence (AI) capacity demands as we anticipate what we may need for the next generations of foundational research and product development. While we are not providing guidance for years beyond 2024, we expect our ambitious long-term AI research and product development efforts will require growing infrastructure investments beyond this year." Based on this text what are

In [31]:
query = base_prompt_template.format(query=summary3)

modelInput = tokenizer(query, return_tensors="pt").to("cuda")

outputs = model.generate(**modelInput, max_new_tokens = 500, do_sample = True)
print(tokenizer.batch_decode(outputs)[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> 
    You are a financial analyst designed to answer questions about business and finance. Your job is to reply to questions about finance topics and provide advice.

    Instruction: "Product Liability We recorded liabilities of $596 million and $561 million in Accrued liabilities and Other liabilities at September 30, 2023 and December 31, 2022 for the expected cost of all known product liability claims, plus an estimate of the expected cost for product liability claims that have already been incurred and are expected to be filed in the future for which we are self-insured. It is reasonably possible that our accruals for product liability claims may increase in future periods in material amounts, although we cannot estimate a reasonable range of incremental loss based on currently available information. We believe that any judgment against us involving our products for actual damages will be adequately covered by our recorded accruals and, where applicable, excess liability insura